# Using BLE RSSI Values to Classify Proximity with Deep Neural Networks

In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

## Data Preparation and Preprocessing 

In [2]:
train_labels = []
train_samples = []

In [3]:
for i in range(1000):
    random_younger = randint(-98,-74)
    train_samples.append(random_younger)
    train_labels.append(0)
    
    random_older = randint(-73,-40)
    train_samples.append(random_older)
    train_labels.append(1)

In [4]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels, train_samples = shuffle(train_labels, train_samples)

In [5]:
print(train_samples)

[-85 -98 -92 ... -40 -91 -81]


In [6]:
# scaler = MinMaxScaler(feature_range=(0,1))
# scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

In [7]:
# print(scaled_train_samples)

## Simple tf.keras Sequential Model

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [9]:
physial_devices = tf.config.experimental.list_physical_devices('GPU')
print("Avaialble: ", len(physial_devices))


Avaialble:  0


In [10]:
model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'), # irst hidden layer - 16 neurons
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(x=train_samples, y=train_labels, validation_split=0.1, batch_size=10, epochs=30, shuffle=True, verbose=2)

Epoch 1/30
180/180 - 0s - loss: 0.6723 - accuracy: 0.5017 - val_loss: 0.6710 - val_accuracy: 0.4850
Epoch 2/30
180/180 - 0s - loss: 0.6652 - accuracy: 0.5056 - val_loss: 0.6643 - val_accuracy: 0.4850
Epoch 3/30
180/180 - 0s - loss: 0.6618 - accuracy: 0.5017 - val_loss: 0.6558 - val_accuracy: 0.4850
Epoch 4/30
180/180 - 0s - loss: 0.6541 - accuracy: 0.5089 - val_loss: 0.6496 - val_accuracy: 0.4850
Epoch 5/30
180/180 - 0s - loss: 0.6483 - accuracy: 0.5256 - val_loss: 0.6431 - val_accuracy: 0.4850
Epoch 6/30
180/180 - 0s - loss: 0.6429 - accuracy: 0.5128 - val_loss: 0.6390 - val_accuracy: 0.6050
Epoch 7/30
180/180 - 0s - loss: 0.6357 - accuracy: 0.5428 - val_loss: 0.6308 - val_accuracy: 0.5600
Epoch 8/30
180/180 - 0s - loss: 0.6351 - accuracy: 0.5589 - val_loss: 0.6241 - val_accuracy: 0.5600
Epoch 9/30
180/180 - 0s - loss: 0.6260 - accuracy: 0.5756 - val_loss: 0.6212 - val_accuracy: 0.7250
Epoch 10/30
180/180 - 0s - loss: 0.6185 - accuracy: 0.5917 - val_loss: 0.6176 - val_accuracy: 0.8100

## Process Test Data

In [14]:
test_labels = []
test_samples = []

In [15]:
for i in range(200):
    random_younger = randint(-98,-74)
    test_samples.append(random_younger)
    test_labels.append(0)
    
    random_older = randint(-73,-40)
    test_samples.append(random_older)
    test_labels.append(1)

In [16]:
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)
test_labels, test_samples = shuffle(test_labels, test_samples)

In [17]:
# print(test_samples)

In [18]:
# scaler = MinMaxScaler(feature_range=(0,1))
# scaled_test_samples = scaler.fit_transform(test_samples.reshape(-1,1))

In [19]:
# print(scaled_test_samples)

## Predict

In [20]:
prediction = model.predict(x=test_samples, batch_size=10, verbose=0)

In [21]:
print(prediction[:10])

[[0.49714828 0.50285167]
 [0.7390814  0.2609186 ]
 [0.56260556 0.43739444]
 [0.39983955 0.6001604 ]
 [0.5946731  0.4053269 ]
 [0.5300058  0.46999416]
 [0.513592   0.486408  ]
 [0.6852596  0.31474042]
 [0.7261968  0.2738032 ]
 [0.7390814  0.2609186 ]]


In [22]:
rounded_prediction = np.argmax(prediction, axis=-1)

In [23]:
print(rounded_prediction[:10])

[1 0 0 1 0 0 0 0 0 0]


In [28]:
model.predict([-100])

array([[0.797402  , 0.20259799]], dtype=float32)

In [24]:
from tensorflow import lite

In [25]:
converter = lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("linear.tflite", "wb").write(tfmodel)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\KAVINDA\AppData\Local\Temp\tmpue1iyn_5\assets


4248